In [ ]:
import cython 
import numpy as np
%load_ext cython

In [ ]:
%%cython -a
import numpy as np
cimport numpy as npc
from scipy.sparse.linalg._expm_multiply import LazyOperatorNormInfo, _fragment_3_1

# a function that prepares a expm_multiply_simple_core call by determining the number of iterations and matrix multiplications
cpdef expm_multiply_prepare(double complex[:,::1] A0, int m, double tol=2**-53):
    """
    Prepare the computation of the action of the matrix exponential of A on B.

    Parameters
    ----------
    A : transposable linear operator
        The operator whose exponential is of interest.
    m : number of wavevectors to use (B will be n x m)
    t : float (default=1.0)
    References
    ----------
    .. [1] Awad H. Al-Mohy and Nicholas J. Higham (2011)
           "Computing the Action of the Matrix Exponential,
           with an Application to Exponential Integrators."
           SIAM Journal on Scientific Computing,
           33 (2). pp. 488-511. ISSN 1064-8275
           http://eprints.ma.man.ac.uk/1591/

    .. [2] Nicholas J. Higham and Awad H. Al-Mohy (2010)
           "Computing Matrix Functions."
           Acta Numerica,
           19. 159-208. ISSN 0962-4929
           http://eprints.ma.man.ac.uk/1451/
    """
    # Check the inputs.
    A = np.array(A0)
    print(A.shape)
    if A.shape[0] != A.shape[1]:
        raise ValueError('expected A to be like a square matrix')
    ident = np.eye(A.shape[0], A.shape[1], dtype=np.complex128)
    n = A.shape[0]
    traceA =  np.array(A).trace()
    mu = traceA / float(n)
    A = A - mu * ident
    A_1_norm = np.linalg.norm(A, 1)  
    if A_1_norm == 0:
        m_star, s = 0, 1
    else:
        norm_info = LazyOperatorNormInfo(A, A_1_norm=t*A_1_norm)
        m_star, s = _fragment_3_1(norm_info, m, tol=tol)
    return m_star, s, np.real(mu)

In [ ]:
d = 4
A = np.random.rand(d,d) + 1j*np.random.rand(d,d)
m_star, s, mu = expm_multiply_prepare(A, 1)
print(m_star, s, mu)

In [ ]:
A.shape

In [ ]:
%%cython -a
cimport cython
from libc.math cimport exp
from scipy.linalg.cython_blas cimport zgemm, zaxpy, zscal
from unipolator.exp_and_log cimport copy_pointer 
import numpy as np
cimport numpy as np

cdef void c_mat_add_pointer(double complex *a0, double complex *b0, int nn) nogil: # A = A+B*c
    cdef int incz = 1
    cdef double complex c = 1.0 + 0.0j
    zaxpy(&nn, &c, b0, &incz, a0, &incz)

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cpdef double norm_inf_complex(double complex[:, ::1] B) nogil:
    cdef Py_ssize_t i, j, n_rows, n_cols
    cdef double max_sum, row_sum
    
    n_rows = B.shape[0]
    n_cols = B.shape[1]
    
    max_sum = 0.0
    for i in range(n_rows):
        row_sum = 0.0
        for j in range(n_cols):
            row_sum += abs(B[i, j])
        if row_sum > max_sum:
            max_sum = row_sum
    return max_sum

cdef void MM_cdot_pointer_v_scaled(double complex *a0, double complex *v0, double complex *c0, double complex alpha, int n, int m) nogil:
    # matrix multiply 2 matrices A (n x n) and B (n x m)
    cdef char *ori = 'n'
    #cdef double complex alpha = 1.0
    cdef double complex beta = 0.0
    zgemm(ori, ori, &m, &n, &n, &alpha, v0, &m, a0, &n, &beta, c0, &m)


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cpdef void expm_multiply_cython(double complex[:, ::1] A, double complex[:, ::1] B, double complex[:, ::1] F, double complex[:, ::1] G, double t, double mu, int m_star, int s) nogil:
    # A (nxn), B (nxm), F (nxm), G (nxm)
    # s is the number of iterations to perform, 
    # Set tolerance to machine precision
    cdef double complex *a0 = &A[0, 0]
    cdef double complex *b0 = &B[0, 0]
    cdef double complex *f0 = &F[0, 0]
    cdef double complex *g0 = &G[0, 0]
    cdef int temp_int, i, j
    cdef int incz = 1
    cdef double tol = 2 ** -53 
    cdef int n = B.shape[0]
    cdef int m = B.shape[1]
    cdef int nm = n * m
    cdef double factor
    cdef double complex coeff = 1.0
    cdef double complex eta = exp(t*mu / float(s))  # Compute scaling factor eta
    cdef double complex a, b
    
    copy_pointer(b0, f0, nm)  # Initialize F as B
    # Perform s iterations
    for i in range(s):
        c1 = norm_inf_complex(B)            # Compute infinity norm of B
        for j in range(m_star):
            temp_int = s*(j+1)
            factor = <double> float(temp_int)
            coeff = t / factor   #.real
            #G = A @ B 
            # preform matrix multiplication G = A @ B 
            MM_cdot_pointer_v_scaled(a0, b0, g0, coeff, n, m)  # Update B ( B = coeff * A.dot(B) ) -> switch every iteration between B and G
            # switch pointers of B and G    #Old Approach: C = B; B = G; G = C
            copy_pointer(g0, b0, nm)
            #B, G = G, B
            c2 = norm_inf_complex(B)        # Compute norm of updated B
            c_mat_add_pointer(f0, b0, nm)   # Update F ( F= F + B )

            if c1 + c2 <= tol * norm_inf_complex(F):  # Check if tolerance is reached
                break
            c1 = c2
        zscal(&nm, &eta, f0, &incz)
        # c_mat_scale(F, eta)  # Scale F --> assumes square matrix
        #B = F
        copy_pointer(f0, b0, nm)  # Copy F to B
    #return F  # Return final value of F


In [ ]:
def _exact_inf_norm(A):
    # A compatibility function which should eventually disappear.
    return np.linalg.norm(A, np.inf)
    
def _expm_multiply_simple_core(A, B, t, mu, m_star, s, tol=None, balance=False):
    tol = 2 ** -53
    F = B
    eta = np.exp(t*mu / float(s))
    for i in range(s):
        c1 = _exact_inf_norm(B)
        for j in range(m_star):
            coeff = t / float(s*(j+1))
            B = coeff * A.dot(B)
            c2 = _exact_inf_norm(B)
            F = F + B

            if c1 + c2 <= tol * _exact_inf_norm(F):
                break
            c1 = c2
        F = eta * F
        B = F
    return F

In [ ]:
# Test the exponentiation
import discrete_quantum as dq
import numpy as np
#dim = 128
for dim in [2, 4]:#, 8, 16, 32, 64, 128, 256, 512, 1024]:
    m = 1
    H = dq.randH_gauss(dim)
    t = 1.0
    phi = dq.rand_wf(dim, 1)
    phi1 = phi.copy()
    phi_new, phi3 = np.zeros_like(phi), np.zeros_like(phi)

    # prepare exponentiation
    m_star, s, mu = expm_multiply_prepare(-1j*H, m, t=t)
    print('_'*5 + 'Dim: ' + str(dim) + ' #x=' + str(m_star * s) + '_'*20)
    
    res2 = _expm_multiply_simple_core(-1j*H, phi1, t, mu, m_star, s)  
    print(phi[0])
    expm_multiply_cython(-1j*H, phi, phi_new, phi3, t, mu, m_star, s)
    print(phi[0])
    
    #phi_new = np.array(res)
    phi_new2 = np.array(res2)
    phi_new_expmh = dq.expmH(H, t) @ phi1
    print('  Difference expmH vs. cython: ', np.linalg.norm(phi_new - phi_new_expmh))
    print('  Difference cython vs. scipy: ', np.linalg.norm(phi_new - phi_new2))

In [ ]:
# time the executions of the exponentiation
import timeit
import matplotlib.pyplot as plt
time_scipy, time_cython, time_expmH = [], [], []
dims = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
reps = 100
for dim in dims:
    print('_'*5 + 'Dim: ' + str(dim) + ' '+ '_'*20)
    m = 1
    H = dq.randH_gauss(dim)
    t = 1.0
    phi = dq.rand_wf(dim, 1)
    phi1 = phi.copy()
    phi_new, phi3 = np.zeros_like(phi), np.zeros_like(phi)

    # prepare exponentiation
    m_star, s, mu = expm_multiply_prepare(-1j*H, m, t=t)
    time_scipy.append(timeit.timeit(lambda: _expm_multiply_simple_core(-1j*H, phi, t, mu, m_star, s), number=reps))
    time_cython.append(timeit.timeit(lambda: expm_multiply_cython(-1j*H, phi, phi_new, phi3, t, mu, m_star, s), number=reps))
    time_expmH.append(timeit.timeit(lambda: dq.expmH(H, t) @ phi1, number=reps))
# transform into arrays
time_scipy = np.array(time_scipy)/reps
time_cython = np.array(time_cython)/reps
time_expmH = np.array(time_expmH)/reps
dims = np.array(dims)
# plot the results
fig, ax = plt.subplots(1, 1, figsize=(10,10))
ax.loglog(dims, time_scipy, label='scipy')
ax.loglog(dims, time_cython, label='cython')
ax.loglog(dims, time_expmH, label='expmH')
# add legend
ax.legend()
ax.set_xlabel('Dimension')
ax.set_ylabel('Time [s]')

In [ ]:
%%cython -a
cimport cython
# import numpy
import numpy as np
from scipy.sparse.linalg._expm_multiply import LazyOperatorNormInfo, _fragment_3_1

# a function that prepares a expm_multiply_simple_core call by determining the number of iterations and matrix multiplications
cpdef expm_multiply_prepare(A, m, t=1.0):
    """
    Prepare the computation of the action of the matrix exponential of A on B.

    Parameters
    ----------
    A : transposable linear operator
        The operator whose exponential is of interest.
    m : number of wavevectors to use (B will be n x m)
    t : float (default=1.0)
    References
    ----------
    .. [1] Awad H. Al-Mohy and Nicholas J. Higham (2011)
           "Computing the Action of the Matrix Exponential,
           with an Application to Exponential Integrators."
           SIAM Journal on Scientific Computing,
           33 (2). pp. 488-511. ISSN 1064-8275
           http://eprints.ma.man.ac.uk/1591/

    .. [2] Nicholas J. Higham and Awad H. Al-Mohy (2010)
           "Computing Matrix Functions."
           Acta Numerica,
           19. 159-208. ISSN 0962-4929
           http://eprints.ma.man.ac.uk/1451/
    """
    # Check the inputs.
    if len(A.shape) != 2 or A.shape[0] != A.shape[1]:
        raise ValueError('expected A to be like a square matrix')
    ident = np.eye(A.shape[0], A.shape[1], dtype=A.dtype)
    n = A.shape[0]
    traceA =  A.trace()
    mu = traceA / float(n)
    A = A - mu * ident
    A_1_norm = np.linalg.norm(A, 1)  
    if t*A_1_norm == 0:
        m_star, s = 0, 1
    else:
        norm_info = LazyOperatorNormInfo(t*A, A_1_norm=t*A_1_norm)
        m_star, s = _fragment_3_1(norm_info, m)
    return m_star, s, mu